#### Initialzation

In [1]:
if True: # enable folding code
    if False:
        from julia.api import Julia
        jl = Julia(compiled_modules=False)

    import julia; julia.install(quiet=True)
    from julia import Main

    import holoviews as hv; hv.extension('bokeh', logo=False)
    import panel as pn; pn.extension()
    import param

    import itikz
    from itikz import nicematrix as nM

    import numpy as np
    import pandas as pd
    from math import atan2, degrees

    from IPython.display import display, Math

    np.set_printoptions(precision=3, suppress=True)

    try:
        from scipy.linalg import norm, inv, cholesky, qr, eig, eigh, qz, ordqz
        HAVE_SCIPY = True
    except Exception as e:
        HAVE_SCIPY = False
        print("SciPy not available;", e)

%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [2]:
%%julia
using Pkg
gla_dir = "../GenLinAlgProblems"
Pkg.activate(gla_dir)
using GenLinAlgProblems, LinearAlgebra, BlockArrays, LaTeXStrings, Latexify, SymPy, Random

using PyCall
itikz = pyimport("itikz")
nM    = pyimport("itikz.nicematrix");

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


In [3]:
%%julia
# support functions used to display objects for discussion
"""
    classify_gsvd(F; tol = 1e-12)

Given a GSVD factorization object `F` (as returned by `svd(A,B)`),
recover the number of
  • pure-A   (alpha=1, beta=0)
  • mixed    (0<alpha<1, 0<beta<1)
  • pure-B   (alpha=0, beta=1)
  • nullspace (alpha=0, beta=0)

using the LAPACK structural constraints:

  - indices 1:k       → pure-A
  - indices k+1:k+l   → mixed OR pure-B
  - indices > k+l     → nullspace

Returns a named tuple.
"""
function classify_gsvd(F; tol=1e-12, show=true)
    alpha = F.alpha
    beta  = F.beta
    k     = F.k               # pure-A
    l     = F.l               # mixed + pure-B
    N     = length(alpha)
    r     = k + l             # rank

    # index partitions
    pureA_idx = 1:k
    mid_idx   = k+1 : k+l
    null_idx  = k+l+1 : N

    # pure A = exactly the first k indices by LAPACK definition
    npureA = k

    # pure B: among mid block, (alpha ≈ 0, beta ≈ 1)
    npureB = count(i -> abs(alpha[i]) < tol && abs(beta[i] - 1) < tol, mid_idx)

    # mixed: remaining mid-block entries where both alpha, beta are nonzero
    nmixed = l - npureB

    # nullspace: trailing (0,0) entries
    nnull = count(i -> abs(alpha[i]) < tol && abs(beta[i]) < tol, null_idx)

    if show
        py_show( L"pureA = ",        npureA,
                 L",\;\; mixed = ",  nmixed,
                 L",\;\ pureB = ",   npureB,
                 L",\;\;n_0 = ",     nnull )
    end

    return (
        n_A  = npureA,
        n_M  = nmixed,
        n_B  = npureB,
        n_0  = nnull,
        rank = r,
        n    = N
    )
end

"""
    res = function gsvd(A,B)

utility function to augmment ths svd(routine) with additional information
returns
* res.F   the GSVD standard structure
* res.X   the domain basis matrix
* res.C   the C matrix augmented by the n_0 block  (size(C) == size(A))
* res.S   the S matrix augmented by the n_0 block  (size(S) == size(B))
* res.sz  the sizes of the subspaces (n_A, n_M, n_B, n_0) and the combined rank
"""
function naive_gsvd(A,B)
    F = svd(A, B)
    U,V,Q,C,S,R = F
    H = R*Q'

    N = nullspace([A; B])
    X = size(N, 2) == 0 ? pinv(H) : [pinv(H) N]

    sz = classify_gsvd(F; show=false)
    if sz.n_0 > 0
        C = [C zeros(size(C,1), sz.n_0)]
        S = [S zeros(size(S,1), sz.n_0)]
    end

    m_A,N  = size(A)
    m_B  = size(B,1)

    spec = [ sz.n_A, sz.n_M, sz.n_B, sz.n_0]

    return (
        alpha=F.alpha, beta=F.beta,
        U=BlockArray(U, [size(U,1)], [sz.n_A, sz.n_M, size(U,2)-sz.n_A-sz.n_M] ),
        V=BlockArray(V, [size(V,1)], [sz.n_M, sz.n_B, size(V,2)-sz.n_M-sz.n_B] ),
        X=BlockArray(X, [size(A,2)], spec ),
        C=BlockArray(C, [m_A],       spec ),
        S=BlockArray(S, [m_B],       spec ),
        sz=sz
    )
end
function show_X_U_and_V( X::BlockArray, U::BlockArray, V::BlockArray, sz::NamedTuple; digits=3)
    function show_X()
        parts = String[]
        n_cols = size(X, 2)
        N_used = sz.n_A + sz.n_M
        if sz.n_A > 0 push!(parts, "X_A") end
        if sz.n_M > 0 push!(parts, "X_M") end
        if sz.n_B > 0 push!(parts, "X_B") end
        if sz.n_0 > 0 push!(parts, "X_0") end

        return LaTeXString( "(" * join(parts,"\\; ") * ") = "), X
    end
    function show_U()
        parts = String[]
        n_cols = size(U, 2)
        N_used = sz.n_A + sz.n_M
        if sz.n_A > 0 push!(parts, "U_A") end
        if sz.n_M > 0 push!(parts, "U_M") end
        if n_cols > N_used push!(parts, "U_0") end

        return LaTeXString( "(" * join(parts,"\\; ") * ") = "), U
    end
    function show_V()
        parts = String[]
        n_cols = size(V, 2)
        N_used = sz.n_B + sz.n_M
        if sz.n_M > 0 push!(parts, "V_M") end
        if sz.n_B > 0 push!(parts, "V_B") end
        if n_cols > N_used push!(parts, "V_0") end

        return LaTeXString( "(" * join(parts,"\\; ") * ") = "), V
    end
    py_show( "X, U and V matrices", color="blue")
    py_show( show_X()..., L",\quad ", show_U()..., L",\quad ", show_V()..., number_formatter=x->round(x,digits=digits))
end
function show_C_and_S( C::BlockArray, S::BlockArray, sz::NamedTuple; digits=3)
    function highlight_C(x, i, j, formatted; sz=sz)
        if 1 ≤ i ≤ sz.n_A+sz.n_M && i == j
            return "\\textcolor{red}{" * formatted * "}"
        else
            return formatted
        end
    end

    function highlight_S(x, i, j, formatted; sz=sz)
        if 1 ≤ i ≤ sz.n_M + sz.n_B && i + sz.n_A == j
            return "\\textcolor{red}{" * formatted * "}"
        else
            return formatted
        end
    end
    py_show( "C and S matrices", color="blue")
    py_show( L"C = ", (C, per_element_style=highlight_C), L",\quad S = ", (S, per_element_style=highlight_S), number_formatter=x->round(x,digits=digits))
end
function show_X_basis( A,B,res; digits=3)
    c(A) = (m=A,color="red")
    sz=res.sz
    XA,XM,XB,X0 = res.X[:, 1:sz.n_A],
                  res.X[:,sz.n_A+1:sz.n_A+sz.n_M],
                  res.X[:,sz.n_A+sz.n_M+1:sz.n_A+sz.n_M+sz.n_B],
                  res.X[:,sz.n_A+sz.n_M+sz.n_B+1:end]
    py_show( "A X and B X", color="blue")

    if sz.n_A > 0 py_show(L"X_A = ", XA, L"\quad  A X_A = ",   A*XA,  L",\quad B X_A = ", c(B*XA), number_formatter=x->round_value(x,digits)) end
    if sz.n_M > 0 py_show(L"X_M = ", XM, L"\quad  A X_M = ",   A*XM,  L",\quad B X_M = ",    B*XM, number_formatter=x->round_value(x,digits)) end
    if sz.n_B > 0 py_show(L"X_B = ", XB, L"\quad  A X_B = ", c(A*XB), L",\quad B X_B = ",    B*XB, number_formatter=x->round_value(x,digits)) end
    if sz.n_0 > 0 py_show(L"X_0 = ", X0, L"\quad  A X_0 = ", c(A*X0), L",\quad B X_0 = ", c(B*X0), number_formatter=x->round_value(x,digits)) end
end

using IJulia
function show_all(A,B,res; digits=3)
    py_show(L"A = ", A, L",\quad B = ", B, number_formatter=x->round_value(x,digits))
    py_show( "sizes: ", string(res.sz), color="blue")
    show_C_and_S(res.C, res.S, res.sz; digits=digits)
    show_X_U_and_V(res.X,res.U,res.V,res.sz; digits=digits)
    show_X_basis(A,B,res; digits=digits)
end
;

In [4]:
def construct_gsvd_pair(n_A, n_M, n_B, n_0, m_A=None, m_B=None, seed=None):
    """
    Construct A = U C Xinv and B = V S Xinv, using specified GSVD subspace sizes.

    Parameters
    ----------
    n_A, n_M, n_B, n_0 : int
        Subspace dimensions: exclusive A, mixed, exclusive B, nullspace.
    m_A, m_B : int, optional
        Number of rows in A and B. Defaults to minimal required sizes.
    seed : int, optional
        Random seed for reproducibility.

    Returns
    -------
    A : ndarray (m_A x n)
    B : ndarray (m_B x n)
    Xinv : ndarray (n x n), invertible matrix used in construction
    U : ndarray (m_A x m_A), orthogonal
    V : ndarray (m_B x m_B), orthogonal
    C : ndarray (m_A x n), GSVD block structure
    S : ndarray (m_B x n), GSVD block structure
    """
    if seed is not None:
        np.random.seed(seed)

    n = n_A + n_M + n_B + n_0  # total domain size
    m_A = m_A or (n_A + n_M)
    m_B = m_B or (n_B + n_M)

    # Step 1: Invertible matrix Xinv
    while True:
        Xinv = np.random.randn(n, n)
        if np.linalg.matrix_rank(Xinv) == n:
            break

    # Step 2: Orthogonal U (m_A x m_A), V (m_B x m_B)
    U, _ = qr(np.random.randn(m_A, m_A))
    V, _ = qr(np.random.randn(m_B, m_B))

    # Step 3: Construct C and S with correct block structure
    C = np.zeros((m_A, n))
    S = np.zeros((m_B, n))

    col = 0

    # --- Exclusive A (pure A)
    if n_A > 0:
        C[:, col:col + n_A] = np.random.randn(m_A, n_A)
        # S remains zero
        col += n_A

    # --- Mixed directions
    if n_M > 0:
        theta_deg = np.random.uniform(1, 89, n_M)
        theta_rad = np.radians(theta_deg)
        for i in range(n_M):
            C[:, col] = np.cos(theta_rad[i]) * np.random.randn(m_A)
            S[:, col] = np.sin(theta_rad[i]) * np.random.randn(m_B)
            col += 1

    # --- Exclusive B (pure B)
    if n_B > 0:
        S[:, col:col + n_B] = np.random.randn(m_B, n_B)
        # C remains zero
        col += n_B

    # --- Nullspace: n_0 columns of all zeros already handled by initialization

    # Step 4: Construct A = U @ C @ Xinv, B = V @ S @ Xinv
    A = U @ C @ Xinv
    B = V @ S @ Xinv

    return A, B, Xinv, U, V, C, S

# 

<div style="float:center; width:100%; text-align:center;">
  <strong style="height:100px; color:darkred; font-size:40px;">
    GSVD Algorithms
  </strong>
</div>
<div></div>

# 1. Introduction

The previous notebook [**GSVD_intro.ipynb**](GSVD_intro.ipynb) introduced the Generalized SVP (GSVP.  
In the **GSVD**, a pair of **rectangular matrices**
$\; A\in\mathbb{R}^{m_A\times n}, B\in\mathbb{R}^{m_B\times n}$  
is reduced to structured forms  
$\qquad A = U\ C\ X^{-1}, \quad B = V\ S\ X^{-1},$  
with a common **invertible** right factor $X$ and separate **orthogonal** left factors $U$ and $V$.

The matrices $C$ and $S$ are the respective representations of $A$ and $B$ using the basis matrices $X,U$ and $V$.